In [ ]:
import numpy as np
import import_ipynb
from scipy.sparse import lil_matrix
from fem_processing import gaussian_quadrature as gauss_quad

# `local_matrices()`

In [55]:
def local_matrices(cell):
    # Initialize the local matrices
    Ne = len(cell['conn'])
    Se = np.zeros((Ne, Ne), dtype='complex128') # Matriz de rigidez do elemento
    Me = np.zeros((Ne, Ne), dtype='complex128') # Matriz de massa do elemento
    be = np.zeros((Ne, 1), dtype='complex128')  # Vetor de carga do elemento

    # Obter os pontos e pesos da quadratura de ordem 4
    points, weights = gauss_quad.general_1d(n=4)
    for xi, wk in zip(points, weights):
        # Material properties
        pe = cell['p(x)']
        qe = cell['q(x)']
        fe = cell['f(x)']

        # Shape functions and derivatives
        phi = np.array([[1 - xi], [xi]])
        grad_phi = np.array([[-1, 1]])
        
        # Jacobian
        Je = cell['geo']['jacobian'] 
        
        # Determinante do Jacobiano
        Jdet, Jinv = abs(Je), 1/Je
        
        # Matriz de rigidez
        Se += (Jinv * grad_phi).T @ (pe * Jinv * grad_phi) * Jdet * wk

        # Produto externo para criar a matriz de massa M_ij 
        # phi @ phi.T = phi.T * phi = [[phi1*phi1, phi1*phi2], [phi2*phi1, phi2*phi2]]
        Me += qe * phi.T * phi * Jdet * wk

        # Element right-hand-side vector
        be += fe * phi * Jdet * wk
    
    return Se, Me, be

# `global_matrices()`

In [56]:
def global_matrices(mesh_data):
    # Inicializa a matriz global como uma matriz esparsa zero (tamanho NxN)
    Nnodes = len(mesh_data['nodes'])
    Sg = lil_matrix((Nnodes, Nnodes), dtype='complex128')
    Mg = lil_matrix((Nnodes, Nnodes), dtype='complex128')
    bg = lil_matrix((Nnodes, 1), dtype='complex128')

    # Início do processo de montagem
    for cell in mesh_data['cell'].values(): 
        Se, Me, be = local_matrices(cell)

        # loop sobre os nós locais de cada elemento
        for i, ig in enumerate(cell['conn']):
            ig = int(ig)-1
            for j, jg in enumerate(cell['conn']):
                jg = int(jg)-1
                Sg[ig, jg] += Se[i, j]
                Mg[ig, jg] += Me[i, j]
                
            # preenche o vetor global b
            bg[ig, 0] += be[i, 0]

    return Sg, Mg, bg

# `apply_simple_abc()`

In [57]:
def apply_simple_abc(Sg, mesh_data):
    for key, node in mesh_data['nodes'].items():
        # Verifica se o nó é do tipo ABC
        if node['bc']['type'] == 'ABC':  
            e = int(key) - 1
            print(f"Applying ABC at node {key} with value {node['bc']['value']}")
            Sg[e, e] += node['bc']['value']

    return Sg

# `apply_simple_dirichlet()`

In [58]:
def apply_simple_dirichlet(Sg, Mg, bg, mesh_data):
    for key, node in mesh_data['nodes'].items():
        # Verifica se o nó é do tipo Dirichlet
        if node['bc']['type'] == 'Dirichlet':  
            e = int(key) - 1
            print(f"Applying Dirichlet at node {key} with value {node['bc']['value']}")

            # Zera a linha e coluna correspondente ao nó de Dirichlet
            Sg[e, :] = 0
            Mg[e, :] = 0
            
            # Define o valor 1 na diagonal para evitar singularidade
            Sg[e, e] = 0.5
            Mg[e, e] = 0.5
            
            # Ajusta o vetor de forças com o valor de Dirichlet
            bg[e, 0] = node['bc']['value']

    return Sg, Mg, bg

# `apply_simple_bcs()`

In [59]:
def apply_simple_bcs(Sg, Mg, bg, mesh_data):
    for key, node in mesh_data['nodes'].items():
        e = int(key) - 1
        # Verifica se o nó é do tipo ABC
        if node['bc']['type'] == 'ABC':  
            print(f"Applying ABC at node {key} with value {node['bc']['value']}")
            Sg[e, e] += node['bc']['value']
            
        # Verifica se o nó é do tipo Dirichlet
        elif node['bc']['type'] == 'Dirichlet':  
            print(f"Applying Dirichlet at node {key} with value {node['bc']['value']}")

            # Zera a linha e coluna correspondente ao nó de Dirichlet
            Sg[e, :] = 0
            Mg[e, :] = 0
            
            # Define o valor 1 na diagonal para evitar singularidade
            Sg[e, e] = 0.5
            Mg[e, e] = 0.5
            
            # Ajusta o vetor de forças com o valor de Dirichlet
            bg[e, 0] = node['bc']['value']

    return Sg, Mg, bg

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2025.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.